In [1]:
from torch.utils.data import Dataset
import torch
from PIL import Image
import pandas as pd
import numpy as np
import os

class MyDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        super().__init__()
        
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'name']
        label = self.df.loc[idx, 'label']

        img = Image.open(os.path.join(self.img_dir, img_name))
        if self.transform is not None:
            img = self.transform(img)
        label = torch.tensor(label, dtype=torch.float32)
        
        return img, label

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import mobilenet_v2

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.backbone = mobilenet_v2(weights='DEFAULT').features
        self.head = nn.Sequential(
            nn.Linear(1280, 128, bias=True),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(128, 1, bias=True)
        )
    
    def forward(self, x):
        with torch.no_grad():
            x = self.backbone(x)
        x = F.avg_pool2d(x, kernel_size=7)
        x = torch.flatten(x, start_dim=1)
        out = self.head(x)
        out = out.squeeze(-1)
        
        return out

In [7]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam, lr_scheduler
from tqdm import tqdm

N_EPOCHS = 30
LR = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, hue=0.1, contrast=0.1),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_set = MyDataset('/kaggle/input/occluded-classification/train.csv', '/kaggle/input/occluded-classification/train', train_transform)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, pin_memory=True, num_workers=4)
test_set = MyDataset('/kaggle/input/occluded-classification/test.csv', '/kaggle/input/occluded-classification/test', test_transform)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False, pin_memory=True, num_workers=4)

net = Net().to(DEVICE)
criterion = BCEWithLogitsLoss(reduction='mean')
optimizer = Adam(net.parameters(), lr=LR, weight_decay=2e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_acc = 0
for epoch in tqdm(range(N_EPOCHS)):
    net.train()
    for imgs, labels in train_loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        
        preds = net(imgs)
        loss = criterion(preds, labels)
        print('loss', loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler.step()
        
    n_corrects = 0
    n_total = 0
    net.eval()
    for imgs, labels in test_loader:
        with torch.no_grad():
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE).int()
            
            preds = net(imgs)
            preds = torch.sigmoid(preds)
            
            preds = (preds > 0.5).int()
            n_corrects += (preds==labels).sum().item()
            n_total += labels.shape[0]
    acc = n_corrects / n_total
    print('Accuracy', acc)
    if acc > best_acc:
        best_acc = acc
        torch.save(net.state_dict(), 'best.pt')
        print('Save best model')


  0%|          | 0/30 [00:00<?, ?it/s]

loss 0.6975126266479492
loss 0.6138490438461304
loss 0.5274912118911743
loss 0.45299601554870605
loss 0.3776739239692688
loss 0.3137471675872803
loss 0.28036341071128845
loss 0.22848838567733765
loss 0.22870251536369324
loss 0.1925293207168579
loss 0.15493369102478027
loss 0.1567322015762329
loss 0.11481891572475433
loss 0.10150741785764694
loss 0.12071577459573746
loss 0.08970075100660324
loss 0.09523215889930725
loss 0.06914585828781128
loss 0.09441111236810684
loss 0.10131168365478516
loss 0.07782498747110367
loss 0.07088179886341095
loss 0.059851400554180145
loss 0.08214394748210907
loss 0.09570236504077911
loss 0.07261134684085846
loss 0.07637092471122742
loss 0.057315558195114136
loss 0.061309024691581726
loss 0.07752428948879242
loss 0.08075021207332611
loss 0.05234060809016228
loss 0.040226273238658905
loss 0.5476911664009094


  3%|▎         | 1/30 [00:32<15:45, 32.60s/it]

Accuracy 0.9619565217391305
Save best model
loss 0.03331450745463371
loss 0.040685299783945084
loss 0.08134031295776367
loss 0.11667314171791077
loss 0.0959329605102539
loss 0.08565390110015869
loss 0.10700250416994095
loss 0.07419125735759735
loss 0.061460383236408234
loss 0.037148550152778625
loss 0.041634418070316315
loss 0.03328271210193634
loss 0.11538273096084595
loss 0.028453068807721138
loss 0.05952465906739235
loss 0.04588601365685463
loss 0.06276113539934158
loss 0.08202172815799713
loss 0.022210121154785156
loss 0.02335028350353241
loss 0.07102979719638824
loss 0.05092582106590271
loss 0.07459160685539246
loss 0.02813188172876835
loss 0.03833569586277008
loss 0.05588702857494354
loss 0.05437257140874863
loss 0.05275065451860428
loss 0.07262995094060898
loss 0.053320106118917465
loss 0.03474438562989235
loss 0.03487009555101395
loss 0.05344325304031372
loss 0.10119077563285828


  7%|▋         | 2/30 [01:04<14:54, 31.95s/it]

...

Accuracy 0.9809782608695652
loss 0.005652287509292364
loss 0.01242223009467125
loss 0.03469013422727585
loss 0.009881109930574894
loss 0.02220626175403595
loss 0.012022537179291248
loss 0.024580588564276695
loss 0.015531701035797596
loss 0.014591651037335396
loss 0.024986479431390762
loss 0.014871813356876373
loss 0.005302192643284798
loss 0.022842247039079666
loss 0.013799715787172318
loss 0.027197327464818954
loss 0.004910686518996954
loss 0.028613576665520668
loss 0.024533383548259735
loss 0.008264154195785522
loss 0.0082020815461874
loss 0.020284375175833702
loss 0.007138810120522976
loss 0.017005449160933495
loss 0.033757708966732025
loss 0.009679866954684258
loss 0.008306203410029411
loss 0.03999900445342064
loss 0.02742307260632515
loss 0.022384092211723328
loss 0.03976849094033241
loss 0.0340503491461277
loss 0.03619394451379776
loss 0.03297136723995209
loss 0.4761165380477905


 97%|█████████▋| 29/30 [15:00<00:30, 30.88s/it]

Accuracy 0.9809782608695652
loss 0.007967008277773857
loss 0.01280718483030796
loss 0.009578405879437923
loss 0.013369184918701649
loss 0.019297650083899498
loss 0.012826632708311081
loss 0.010609208606183529
loss 0.04194413125514984
loss 0.019764885306358337
loss 0.006304935552179813
loss 0.007917508482933044
loss 0.028050361201167107
loss 0.017909616231918335
loss 0.004466834478080273
loss 0.0196678563952446
loss 0.028012588620185852
loss 0.01895834691822529
loss 0.018697403371334076
loss 0.010330851189792156
loss 0.006858587730675936
loss 0.04583778232336044
loss 0.04786085709929466
loss 0.011471390724182129
loss 0.020678937435150146
loss 0.016020517796278
loss 0.010534530505537987
loss 0.014715532772243023
loss 0.009930587373673916
loss 0.006922970060259104
loss 0.04375892132520676
loss 0.026641596108675003
loss 0.015911180526018143
loss 0.009406882338225842
loss 0.00044050836004316807


100%|██████████| 30/30 [15:31<00:00, 31.04s/it]

Accuracy 0.9818840579710145
